In [ ]:
import cv2
from PIL import Image, ImageEnhance
import numpy as np
from numpy import random
import os
from tqdm import tqdm

# Video Loader

This loads any video with .mp4 extension(if want to load video of any other extension then channge the fourcc and the textual extension manually). It loads a colour video and changes it to grayscale with dimentions of 512,512.

In [ ]:
def vid_loader(path):
    frames = []
    cam = cv2.VideoCapture(path)
    fps = cam.get(cv2.CAP_PROP_FPS)
    while(cam.isOpened()): 
        ret,frame = cam.read()
        if ret:
            frame = cv2.resize(frame, (512,512))
            frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
            img = Image.fromarray(frame)
            img = img.rotate(-90)
            frames.append(img)
        else: 
            break
    cam.release()
    cv2.destroyAllWindows()
    num_frames = len(frames)
    videodims = frames[0].size
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    return videodims, fourcc, fps, num_frames, frames

# Video Writer

This function saves the augmented video in .mp4 extension

In [ ]:
def writeVid(op, vid, iteration, fps):
    video = cv2.VideoWriter("augmented/"+str(lbl)+"/"+str(vidd)+"_"+str(op)+str(iteration)+".mp4",fourcc, fps, videodims, 0)
    #print("creating "+str(op)+str(iteration)+".mp4")
    for i in range(0, num_frames):
        video.write(np.array(vid[iteration][i]))
    video.release()

In [ ]:
# Here n are the number of videos to be produced
def aug_rotate(n, degree):
    rotated = []
    for i in range(n):
        angle = random.uniform(-degree, degree)
        rot = [img.rotate(angle) for img in vid]
        rotated.append(rot)
        writeVid("rotate", rotated, i, fps)

def aug_brightness(n, value):
    bright = []
    for i in range(n):
        nits = random.uniform(0.5, value)
        bri = [ImageEnhance.Brightness(img).enhance(nits) for img in vid]
        bright.append(bri)
        writeVid("brightness", bright, i, fps)

def aug_contrast(n, value):
    contrast = []
    for i in range(n):
        nits = random.uniform(1, value)
        cont = [ImageEnhance.Contrast(img).enhance(nits) for img in vid]
        contrast.append(cont)
        writeVid("contrast", contrast, i, fps)

def aug_zoom(n, zoom_factor):
    zoom_list = []
    for i in range(n):
        w, h = videodims[0], videodims[1]
        x = random.uniform(w*0.25, w*0.75)
        y = random.uniform(h*0.25, h*0.75)
        zoom = random.uniform(0.25,zoom_factor)
        zoom2 = zoom * 2
        zoo = [img.crop((x - w / zoom2, y - h / zoom2, x + w / zoom2, y + h / zoom2)).resize((w,h), Image.LANCZOS) for img in vid]
        zoom_list.append(zoo)
        writeVid("zoom", zoom_list, i, fps)

def aug_upsample(n, ups):
    op = "upsample"
    for i in range(n):
        nits = random.uniform(1, ups)
        fps_1 = fps*nits
        video = cv2.VideoWriter("augmented/"+str(lbl)+"/"+str(vidd)+"_"+str(op)+str(i)+".mp4",fourcc, fps_1, videodims, 0)
        #print("creating upsample"+str(i)+".mp4")
        for i in range(0, num_frames):
            video.write(np.array(vid[i]))
        video.release()

def aug_downsample(n, dwns):
    op = "downsample"
    for i in range(n):
        nits = random.uniform(dwns, 1)
        fps_1 = fps*nits
        video = cv2.VideoWriter("augmented/"+str(lbl)+"/"+str(vidd)+"_"+str(op)+str(i)+".mp4",fourcc, fps_1, videodims, 0)
        #print("creating downsample"+str(i)+".mp4")
        for i in range(0, num_frames):
            video.write(np.array(vid[i]))
        video.release()
        

In [ ]:
datadir = ""
categories = {0:"sign0",1:"sign1"}
label = []
for category in tqdm(categories, desc="Augmenting videos"):
    path = os.path.join(datadir,categories[category])
    lbl = categories[category]
    label.append(lbl)
    #print(lbl)
    for vids in os.listdir(path):
        vidd = vids.split('.')[0]
        #print(vidd)
        videodims, fourcc, fps, num_frames, vid = vid_loader(os.path.join(path,vids))
        aug_brightness(5, 10)
        aug_contrast(5, 3) #value should be greater than 1
        aug_rotate(5, 20)
        aug_zoom(5, 1.3) #if value greater than 1(zoom-in and zoom-out) else zoom-out
        aug_upsample(5, 2) #value should be greater than 1
        aug_downsample(5, 0.1) #value should be less than 1(in the lower end)